In [23]:
from langchain_groq import ChatGroq
from langchain.document_loaders.pdf import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms.ollama import Ollama
from langchain.embeddings.ollama import OllamaEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain.vectorstores.pinecone import Pinecone
from langchain_pinecone.vectorstores import PineconeVectorStore
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts.chat import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from dotenv import load_dotenv
import os

In [24]:
load_dotenv()
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

In [86]:
pdf_loader = PyPDFLoader('Dr Sanjay Sachdeva.pdf')
docs = pdf_loader.load()

In [87]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200
)
documents = text_splitter.split_documents(docs)

In [88]:
embeddings = OllamaEmbeddings(
    model = "mxbai-embed-large"
)

In [89]:
from pinecone import Pinecone
os.environ['PINECONE_API_KEY'] = os.getenv('PINECONE_API_KEY')
db = PineconeVectorStore.from_documents(
    documents,
    embeddings,
    index_name = "langchain-index"
)

In [90]:
llm = ChatGroq(
    api_key=os.getenv('GROQ_API_KEY'),
    model='mixtral-8x7b-32768'
)

In [91]:
prompt = ChatPromptTemplate.from_template(
    '''
    Answer the question sbased on the provided context only:
    Please provide the most accurate response based on the question
    Context : {context}
    Question : {input}
    '''
)

In [92]:
document_chain = create_stuff_documents_chain(
    llm,
    prompt
)
retriever = db.as_retriever()

In [93]:
chain = create_retrieval_chain(
    retriever,
    document_chain
)

In [102]:
response = chain.invoke({
    'input':'What is the gender of the pateint?',
})

APIConnectionError: Connection error.

In [101]:
print(response['answer'])

Based on the report provided, the following tests have been conducted on the patient:

1. Leucocyte counts
2. Antinuclear Antibody/Factor (ANA/ANF), Serum (EIA)

These are the only tests explicitly mentioned in the report. Therefore, it is possible that additional tests were ordered but are not included in the provided context.
